In [22]:
import requests
import re
import numpy as np

In [2]:
urlp = re.compile('http.*?"')
p = re.compile('alt=.*?".*?"')

In [18]:
poss_bias_vals = {"extreme left bias" : -3, 
                 "left bias" : -2, 
                 "left biased" : -2, 
                 "left center bias" : -1, 
                 "left center" : -1, 
                 "least biased" : 0, 
                 "right center" : 1, 
                 "right center bias" : 1, 
                 "right bias" : 2, 
                 "right biased" : 2, 
                 "questionable" : 3, 
                 "extreme right bias" : 3, 
                 "right bias -fake news" : 3, 
                 "extreme right bias, fake news" : 3, 
                 "questionable source" : 3}


In [15]:
cred_vals = {"satire" : 0, 
             "credible" : 4, 
             "mostly credible and reliable" : 3, 
             "not always credible" : 2, 
             "not credible" : 0, 
             "not always credible or reliable" : 2, 
             "biased" : 0, 
             "low factual reporting" : 1, 
             "fake news" : 0, 
             "pseudoscience" : 0}


In [13]:
url='https://mediabiasfactcheck.com/?s='
all_vals = {}
with open("./all_uniq_pub.txt") as f:
    for line in f:
        line = line.strip()
        try:
            r = requests.get(url + line)
        except:
            continue
        for j in r.iter_lines():
            if str(j).find("loop-title") != -1:
                u = j
                all_vals[line] = {"Cred" : "", "Bias" : ""}
                furl = urlp.findall(str(u))[0][:-1]
                fu = requests.get(furl)
                n = p.findall(str(fu.content))[:2]
                if (len(n) > 0 and len(n[0].split(' -')) >= 2) :
                    b_val = n[0].split(' - ')[1].replace('"', "").lower()
                    b_val = b_val.strip()
                    if (b_val in poss_bias_vals) :
                        all_vals[line]["Bias"] = b_val
                        print("{} : Bias : {}".format(line, b_val))
                    elif (b_val in cred_vals):
                        all_vals[line]["Cred"] = b_val
                        print("{} : Cred : {}".format(line, b_val))
                    else:
                        print("{} : {}".format(line, n[0]))
                if (len(n) == 2 and len(n[1].split(' - ')) >= 2):
                    b_val = n[1].split(' - ')[1].replace('"', "").lower()
                    b_val = b_val.strip()
                    if (b_val in poss_bias_vals) :
                        all_vals[line]["Bias"] = b_val
                        print("{} : Bias : {}".format(line, b_val))
                    elif (b_val in cred_vals):
                        all_vals[line]["Cred"] = b_val
                        print("{} : Cred : {}".format(line, b_val))
                    else:
                        print("{} : {}".format(line, n[1]))
                break

http://100percentfedup.com : Bias : extreme right bias
http://100percentfedup.com : Cred : not credible
http://allenwestrepublic.com : Bias : extreme right bias
http://allenwestrepublic.com : Cred : not credible
http://alternativemediasyndicate.com : Bias : extreme left bias
http://alternativemediasyndicate.com : Cred : not always credible or reliable
http://bestthings.us : Cred : biased
http://breakingtop.world : Bias : extreme right bias, fake news
http://breakingtop.world : Cred : biased
http://clashdaily.com : Cred : fake news
http://clashdaily.com : Bias : right bias
http://cnn.it : Bias : left biased
http://cnn.it : Cred : mostly credible and reliable
http://conservativearmy88.com : Cred : fake news
http://conservativearmy88.com : Bias : right bias
http://conservativebyte.com : Cred : fake news
http://conservativetribune.com : Bias : questionable
http://conservativetribune.com : Cred : biased
http://dailysnark.com : Cred : satire
http://dailythings.world : Cred : not always credi

In [24]:
with open("./pub_right.txt", "w") as r, open("./pub_left.txt", "w") as l, open("./pub_neutral.txt", "w") as n:
    for i in all_vals:
        if all_vals[i]["Bias"] in poss_bias_vals:
            pv = poss_bias_vals[all_vals[i]["Bias"]]
            if (pv > 0):
                r.write(i + "\t" + str(pv/3.0) + "\n")
            if (pv < 0):
                l.write(i + "\t" + str(-1*pv/3.0) + "\n")
            n.write(i + "\t" + str((3-np.abs(pv))/3.0) + "\n")

In [27]:
with open("./pub_cred.txt", "w") as c:
    for i in all_vals:
        if all_vals[i]["Cred"] in cred_vals:
            pv = cred_vals[all_vals[i]["Cred"]]
            c.write(i + "\t" + str((np.abs(pv))/4.0) + "\n")